<a href="https://colab.research.google.com/github/kaminglui/Domain-Adaptation-with-ME-IIS/blob/main/ME_IIS_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ME-IIS Domain Adaptation (Colab)
Single-pass pipeline to train a source ResNet-50 and adapt with ME-IIS. Edit the Section 3 config cell to switch datasets, domains, and options.

## 0. Setup & GPU check
- Mount Google Drive and choose a working directory.
- Clone/pull the repo into that folder.
- Confirm CUDA availability and show the GPU name.

In [1]:
# Section 0 - Setup & GPU check
import os
from pathlib import Path

USE_DRIVE = True  # Set False to keep everything in /content
DRIVE_ROOT = "/content/drive/MyDrive" if USE_DRIVE else "/content"
WORK_DIR = os.path.join(DRIVE_ROOT, "MEIIS-Colab")
REPO_URL = "https://github.com/kaminglui/Domain-Adaptation-with-ME-IIS.git"
REPO_NAME = "Domain-Adaptation-with-ME-IIS"
REPO_DIR = os.path.join(WORK_DIR, REPO_NAME)

if USE_DRIVE:
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive")

os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)
print("WORK_DIR:", WORK_DIR)

if not os.path.isdir(REPO_DIR):
    print("[Repo] Cloning repository...")
    !git clone {REPO_URL}
else:
    print("[Repo] Repository exists; pulling latest changes...")
    os.chdir(REPO_DIR)
    !git pull
    os.chdir(WORK_DIR)

os.chdir(REPO_DIR)
print("[Repo] Current repo dir:", os.getcwd())

try:
    import torch
    print("torch.cuda.is_available():", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU:", torch.cuda.get_device_name(0))
    else:
        print("GPU not detected - switch runtime to GPU for full runs.")
except ImportError as exc:
    print("PyTorch not installed yet; run Section 1 to install dependencies. GPU check skipped:", exc)


Mounted at /content/drive
WORK_DIR: /content/drive/MyDrive/MEIIS-Colab
[Repo] Repository exists; pulling latest changes...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 48 (delta 31), reused 32 (delta 18), pack-reused 0 (from 0)
Unpacking objects: 100% (48/48), 51.75 KiB | 13.00 KiB/s, done.
From https://github.com/kaminglui/Domain-Adaptation-with-ME-IIS
   3459174..761a61e  main       -> origin/main
Updating 3459174..761a61e
Fast-forward
 ME_IIS_Colab.ipynb                | 559 ++++++++++++++++++++++++++++++++++++++
 README.md                         |  13 +-
 datasets/domain_loaders.py        |  68 ++++-
 models/me_iis_adapter.py          | 100 ++++++-
 run_smoke_tests.py                | 274 +++++++++++++------
 scripts/adapt_me_iis.py           | 102 ++-----
 scripts/demo_me_iis_toy.py        |  71 +++++
 scripts/plot_iis_dynamics.py      |  23 +-
 scripts/run_me_iis_experiments.py

## 1. Install dependencies
Re-run this cell if the Colab runtime restarts.

In [2]:
# Section 1 - Install dependencies (re-run if the runtime restarts)
import os

os.chdir(REPO_DIR)
req_file = "env/requirements_colab.txt" if os.path.exists("env/requirements_colab.txt") else "env/requirements.txt"
print("Installing dependencies from:", req_file)
!pip install -r {req_file}


Installing dependencies from: env/requirements_colab.txt


## 2. Download datasets via KaggleHub
Downloads Office-Home and Office-31 with KaggleHub, locates the canonical roots, and links them under `datasets/` for the scripts.

In [3]:
# Section 2 - Download datasets via KaggleHub
import os
import pathlib

os.chdir(REPO_DIR)
print("Repo dir:", os.getcwd())

try:
    import kagglehub  # type: ignore
except ImportError:
    print("[Data] Installing kagglehub...")
    !pip install kagglehub
    import kagglehub  # type: ignore

def _find_office_home_root(base_dir: pathlib.Path) -> pathlib.Path:
    candidates = [base_dir] + [p for p in base_dir.rglob("*") if p.is_dir()]
    for cand in candidates:
        names = {p.name for p in cand.iterdir() if p.is_dir()}
        if {"Art", "Clipart", "Product"} <= names and any(n.lower().startswith("real") for n in names):
            return cand
    return base_dir

def _find_office31_root(base_dir: pathlib.Path) -> pathlib.Path:
    candidates = [base_dir] + [p for p in base_dir.rglob("*") if p.is_dir()]
    for cand in candidates:
        names = {p.name.lower() for p in cand.iterdir() if p.is_dir()}
        if {"amazon", "dslr", "webcam"} <= names:
            return cand
    return base_dir

print("[Data] Downloading Office-Home (lhrrraname/officehome)...")
office_home_raw = pathlib.Path(kagglehub.dataset_download("lhrrraname/officehome"))
office_home_root = _find_office_home_root(office_home_raw)
print("  Office-Home root:", office_home_root)

print("[Data] Downloading Office-31 (xixuhu/office31)...")
office31_raw = pathlib.Path(kagglehub.dataset_download("xixuhu/office31"))
office31_root = _find_office31_root(office31_raw)
print("  Office-31 root:", office31_root)

datasets_dir = pathlib.Path("datasets")
datasets_dir.mkdir(exist_ok=True)

def _ensure_link(link_path: pathlib.Path, target: pathlib.Path) -> None:
    target = target.resolve()
    if link_path.exists() and not link_path.is_symlink():
        print(f"[Data] {link_path} exists and is not a symlink; leaving as-is.")
        return
    if link_path.is_symlink():
        current = link_path.resolve()
        if current == target:
            print(f"[Data] {link_path} already points to {target}")
            return
        link_path.unlink()
    try:
        link_path.symlink_to(target, target_is_directory=True)
        print(f"[Data] Linked {link_path} -> {target}")
    except OSError as exc:
        print(f"[Data] Could not create symlink {link_path} -> {target}: {exc}")

_ensure_link(datasets_dir / "Office-Home", office_home_root)
_ensure_link(datasets_dir / "Office-31", office31_root)

OFFICE_HOME_ROOT = datasets_dir / "Office-Home"
OFFICE31_ROOT = datasets_dir / "Office-31"
print("[Data] Office-Home DATA_ROOT:", OFFICE_HOME_ROOT)
print("[Data] Office-31 DATA_ROOT:", OFFICE31_ROOT)


Repo dir: /content/drive/MyDrive/MEIIS-Colab/Domain-Adaptation-with-ME-IIS
[Data] Downloading Office-Home (lhrrraname/officehome)...


100%|██████████| 1.06G/1.06G [00:13<00:00, 84.0MB/s]

Extracting files...


  Office-Home root: /root/.cache/kagglehub/datasets/lhrrraname/officehome/versions/1/datasets/OfficeHomeDataset_10072016
[Data] Downloading Office-31 (xixuhu/office31)...


100%|██████████| 75.9M/75.9M [00:00<00:00, 93.6MB/s]

Extracting files...


  Office-31 root: /root/.cache/kagglehub/datasets/xixuhu/office31/versions/1/Office-31
[Data] Linked datasets/Office-Home -> /root/.cache/kagglehub/datasets/lhrrraname/officehome/versions/1/datasets/OfficeHomeDataset_10072016
[Data] Linked datasets/Office-31 -> /root/.cache/kagglehub/datasets/xixuhu/office31/versions/1/Office-31
[Data] Office-Home DATA_ROOT: datasets/Office-Home
[Data] Office-31 DATA_ROOT: datasets/Office-31


## 3. Configuration (single cell with all knobs)
Edit only in this cell to change experiment settings.

In [4]:
# Edit only in this cell to change experiment settings.

# Dataset & domains
DATASET_NAME = "office_home"  # or "office31"
SOURCE_DOMAIN = "Ar"          # e.g. Ar/Cl/Pr/Rw for Office-Home, A/D/W for Office-31
TARGET_DOMAIN = "Cl"
SEED = 0

# Paths (assume we are in the repo root)
DATA_ROOT = "datasets/Office-Home"  # or "datasets/Office-31"

# Source training hyperparameters
NUM_EPOCHS_SRC = 100
BATCH_SIZE_SRC = 32
LR_BACKBONE = 1e-3
LR_CLASSIFIER = 1e-2
WEIGHT_DECAY = 1e-3
NUM_WORKERS = 4
DETERMINISTIC = True  # set True to minimize randomness

# ME-IIS / adaptation hyperparameters
FEATURE_LAYERS = "layer3,layer4,avgpool"
GMM_SELECTION_MODE = "bic"        # "fixed" or "bic"
NUM_LATENT_STYLES = 5             # default components per layer (fixed mode or BIC init)
GMM_BIC_MIN_COMPONENTS = 2        # BIC lower bound
GMM_BIC_MAX_COMPONENTS = 8        # BIC upper bound

# NEW: source_prob_mode
SOURCE_PROB_MODE = "softmax"      # or "onehot" (use GT labels for source constraints)

# NEW: optional per-layer GMM override
# e.g. "layer3:10,layer4:5" or "7,8,9" for 3 layers; leave "" to disable
COMPONENTS_OVERRIDE = ""

# Pseudo-label adaptation (ME-IIS+PL)
USE_PSEUDO_LABELS = False         # default False; set True to use pseudo-labels
PSEUDO_CONF_THRESH = 0.9
PSEUDO_MAX_RATIO = 1.0            # 1.0 = no cap; or e.g. 0.3 to limit pseudo-target count
PSEUDO_LOSS_WEIGHT = 1.0

# IIS configuration
IIS_ITERS = 15
IIS_TOL = 1e-3
ADAPT_EPOCHS = 10
FINETUNE_BACKBONE = False
BACKBONE_LR_SCALE = 0.1
CLASSIFIER_LR = 1e-2
ADAPT_WEIGHT_DECAY = 1e-3

# Optional: force fresh runs instead of auto-resume
FORCE_FRESH_SOURCE_TRAIN = False
FORCE_FRESH_ADAPT = False


## 4. Helper: construct checkpoint paths and cleanup
Lets you intentionally bypass auto-resume by deleting existing checkpoints.

In [5]:
import os

os.chdir(REPO_DIR)

def get_source_ckpt_path():
    return f"checkpoints/source_only_{SOURCE_DOMAIN}_to_{TARGET_DOMAIN}_seed{SEED}.pth"

def get_me_iis_ckpt_path():
    layers_str = FEATURE_LAYERS.replace(",", "-").replace(" ", "")
    return f"checkpoints/me_iis_{SOURCE_DOMAIN}_to_{TARGET_DOMAIN}_{layers_str}_seed{SEED}.pth"

if FORCE_FRESH_SOURCE_TRAIN:
    src_ckpt = get_source_ckpt_path()
    if os.path.exists(src_ckpt):
        os.remove(src_ckpt)
        print("Deleted existing source checkpoint:", src_ckpt)

if FORCE_FRESH_ADAPT:
    adapt_ckpt = get_me_iis_ckpt_path()
    if os.path.exists(adapt_ckpt):
        os.remove(adapt_ckpt)
        print("Deleted existing ME-IIS checkpoint:", adapt_ckpt)


## 5. Train source-only model (full training)
Build the command from the config above and stream output directly in the cell.

In [6]:
import os

det_flag = "--deterministic" if DETERMINISTIC else ""
os.chdir(REPO_DIR)

SRC_CMD = (
    'python scripts/train_source.py \
'
    f'  --dataset_name {DATASET_NAME} \
'
    f'  --data_root "{DATA_ROOT}" \
'
    f'  --source_domain {SOURCE_DOMAIN} \
'
    f'  --target_domain {TARGET_DOMAIN} \
'
    f'  --num_epochs {NUM_EPOCHS_SRC} \
'
    f'  --batch_size {BATCH_SIZE_SRC} \
'
    f'  --lr_backbone {LR_BACKBONE} \
'
    f'  --lr_classifier {LR_CLASSIFIER} \
'
    f'  --weight_decay {WEIGHT_DECAY} \
'
    f'  --num_workers {NUM_WORKERS} \
'
    f'  {det_flag} \
'
    f'  --seed {SEED}'
).strip()

print("Training command:\n", SRC_CMD)
!{SRC_CMD}


Training command:
 python scripts/train_source.py   --dataset_name office_home   --data_root "datasets/Office-Home"   --source_domain Ar   --target_domain Cl   --num_epochs 100   --batch_size 32   --lr_backbone 0.001   --lr_classifier 0.01   --weight_decay 0.001   --num_workers 4   --deterministic   --seed 0
2025-12-10 04:11:20.682777: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-10 04:11:20.699436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765339880.720313    2704 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00

## 6. Run ME-IIS adaptation
Uses the source checkpoint above plus the ME-IIS settings in the config cell.

In [7]:
import os

det_flag = "--deterministic" if DETERMINISTIC else ""
os.chdir(REPO_DIR)

ADAPT_CMD = (
    'python scripts/adapt_me_iis.py \
'
    f'  --dataset_name {DATASET_NAME} \
'
    f'  --data_root "{DATA_ROOT}" \
'
    f'  --source_domain {SOURCE_DOMAIN} \
'
    f'  --target_domain {TARGET_DOMAIN} \
'
    f'  --checkpoint {get_source_ckpt_path()} \
'
    f'  --batch_size {BATCH_SIZE_SRC} \
'
    f'  --num_workers {NUM_WORKERS} \
'
    f'  --feature_layers "{FEATURE_LAYERS}" \
'
    f'  --num_latent_styles {NUM_LATENT_STYLES} \
'
    f'  --gmm_selection_mode {GMM_SELECTION_MODE} \
'
    f'  --gmm_bic_min_components {GMM_BIC_MIN_COMPONENTS} \
'
    f'  --gmm_bic_max_components {GMM_BIC_MAX_COMPONENTS} \
'
    f'  --iis_iters {IIS_ITERS} \
'
    f'  --iis_tol {IIS_TOL} \
'
    f'  --adapt_epochs {ADAPT_EPOCHS} \
'
    f'  {'--finetune_backbone' if FINETUNE_BACKBONE else ''} \
'
    f'  --backbone_lr_scale {BACKBONE_LR_SCALE} \
'
    f'  --classifier_lr {CLASSIFIER_LR} \
'
    f'  --weight_decay {ADAPT_WEIGHT_DECAY} \
'
    f'  --source_prob_mode {SOURCE_PROB_MODE} \
'
    f'  {det_flag} \
'
    f'  --seed {SEED}'
).strip()

if USE_PSEUDO_LABELS:
    ADAPT_CMD += (
        f" --use_pseudo_labels --pseudo_conf_thresh {PSEUDO_CONF_THRESH} "
        f"--pseudo_max_ratio {PSEUDO_MAX_RATIO} --pseudo_loss_weight {PSEUDO_LOSS_WEIGHT}"
    )

if COMPONENTS_OVERRIDE.strip():
    ADAPT_CMD += f' --components_per_layer "{COMPONENTS_OVERRIDE}"'

print("Adaptation command:\n", ADAPT_CMD)
!{ADAPT_CMD}


Adaptation command:
 python scripts/adapt_me_iis.py   --dataset_name office_home   --data_root "datasets/Office-Home"   --source_domain Ar   --target_domain Cl   --checkpoint checkpoints/source_only_Ar_to_Cl_seed0.pth   --batch_size 32   --num_workers 4   --feature_layers "layer3,layer4,avgpool"   --num_latent_styles 5   --gmm_selection_mode bic   --gmm_bic_min_components 2   --gmm_bic_max_components 8   --iis_iters 15   --iis_tol 0.001   --adapt_epochs 10      --backbone_lr_scale 0.1   --classifier_lr 0.01   --weight_decay 0.001   --source_prob_mode softmax   --deterministic   --seed 0
2025-12-10 04:12:01.078006: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-10 04:12:01.094860: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cu

## 7. Optional: experiment driver examples
Leave `RUN_EXPERIMENT_DRIVER = False` for the standard train + adapt path.

In [8]:
RUN_EXPERIMENT_DRIVER = True

if RUN_EXPERIMENT_DRIVER:
    import os
    os.chdir(REPO_DIR)
    det_flag = "--deterministic" if DETERMINISTIC else ""
    EXP_CMD = (
        'python scripts/run_me_iis_experiments.py \
'
        f'  --dataset_name {DATASET_NAME} \
'
        f'  --source_domain {SOURCE_DOMAIN} \
'
        f'  --target_domain {TARGET_DOMAIN} \
'
        '  --experiment_family layers \
'
        f'  --seeds {SEED} \
'
        f'  --base_data_root "{DATA_ROOT}" \
'
        f'  --feature_layers "{FEATURE_LAYERS}" \
'
        f'  --num_latent_styles {NUM_LATENT_STYLES} \
'
        f'  --gmm_selection_mode {GMM_SELECTION_MODE} \
'
        f'  --gmm_bic_min_components {GMM_BIC_MIN_COMPONENTS} \
'
        f'  --gmm_bic_max_components {GMM_BIC_MAX_COMPONENTS} \
'
        f'  --source_prob_mode {SOURCE_PROB_MODE} \
'
        f'  --num_epochs {NUM_EPOCHS_SRC} \
'
        f'  --batch_size {BATCH_SIZE_SRC} \
'
        f'  --num_workers {NUM_WORKERS} \
'
        f'  --iis_iters {IIS_ITERS} \
'
        f'  --iis_tol {IIS_TOL} \
'
        f'  --adapt_epochs {ADAPT_EPOCHS} \
'
        f'  --backbone_lr_scale {BACKBONE_LR_SCALE} \
'
        f'  --classifier_lr {CLASSIFIER_LR} \
'
        f'  --weight_decay {ADAPT_WEIGHT_DECAY} \
'
        f'  {'--finetune_backbone' if FINETUNE_BACKBONE else ''} \
'
        f'  {det_flag} \
'
        f'  --output_csv results/me_iis_layers_{SOURCE_DOMAIN}_to_{TARGET_DOMAIN}_seed{SEED}.csv'
    ).strip()

    if COMPONENTS_OVERRIDE.strip():
        EXP_CMD += f' --components_per_layer "{COMPONENTS_OVERRIDE}"'

    if USE_PSEUDO_LABELS:
        EXP_CMD += (
            f" --pseudo_conf_thresh {PSEUDO_CONF_THRESH} "
            f"--pseudo_max_ratio {PSEUDO_MAX_RATIO} --pseudo_loss_weight {PSEUDO_LOSS_WEIGHT}"
        )

    if det_flag:
        EXP_CMD += f" {det_flag}"

    print("Experiment driver command:\n", EXP_CMD)
    !{EXP_CMD}
else:
    print("Experiment driver is disabled (set RUN_EXPERIMENT_DRIVER = True to run).")


Experiment driver command:
 python scripts/run_me_iis_experiments.py   --dataset_name office_home   --source_domain Ar   --target_domain Cl   --experiment_family layers   --seeds 0   --base_data_root "datasets/Office-Home"   --feature_layers "layer3,layer4,avgpool"   --num_latent_styles 5   --gmm_selection_mode bic   --gmm_bic_min_components 2   --gmm_bic_max_components 8   --source_prob_mode softmax   --num_epochs 100   --batch_size 32   --num_workers 4   --iis_iters 15   --iis_tol 0.001   --adapt_epochs 10   --backbone_lr_scale 0.1   --classifier_lr 0.01   --weight_decay 0.001      --deterministic   --output_csv results/me_iis_layers_Ar_to_Cl_seed0.csv --deterministic
2025-12-10 04:16:58.029089: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-10 04:16:58.046724:

## 8. Notes on outputs
- Source checkpoints: `checkpoints/source_only_*` (auto-resume uses the matching file name).
- ME-IIS checkpoints and IIS weights: `checkpoints/me_iis_*` and `results/me_iis_weights_*.npz`.
- CSV log of source/adapt runs: `results/office_home_me_iis.csv` (dataset column distinguishes Office-Home vs Office-31).
- Experiment driver summaries (optional): `results/me_iis_experiments_summary.csv` or the path you pass via `--output_csv`.
- TensorBoard logs live under `runs/` (`runs/source_only` and `runs/adapt_me_iis`).